# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **sports or athletics** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **sports or athletics**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **sports or athletics**?  For this category we are interested in sporting events or athletics broadly, please feel free to creatively interpret the category when building your research question!

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [ ]:
# Data sources:
# https://datahub.io/core/covid-19
# https://datahub.io/core/finance-vix
# https://datahub.io/core/natural-gas
# https://www.federalreserve.gov/datadownload/Choose.aspx?rel=H15

In [51]:
from io import StringIO
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

def getDataset(data_url):
    # default python header is 'User-Agent': 'python-requests/2.13.0' - a non-browser request
    # change header to avoid HTTP 403 Forbidden error
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    
    contents = requests.get(data_url, headers= headers).text

    df=pd.read_csv(StringIO(contents))
    return df


In [71]:
covid_url = 'https://datahub.io/core/covid-19/r/countries-aggregated.csv'
df_covid = getDataset(covid_url)
df_covid = df_covid[df_covid['Country']=='US'].copy()
df_covid['Date'] = pd.to_datetime(df_covid['Date'])
df_covid.set_index('Date', inplace=True)

# add initial days from 2020-01-01; set counts to 0
missing_days = np.arange(datetime(2020,1,1), df_covid.index.min(), timedelta(days=1)).astype(datetime)
df_complementary = pd.DataFrame({'Country':[0]*len(missing_days), 
                                 'Confirmed':[0]*len(missing_days), 
                                 'Recovered':[0]*len(missing_days), 
                                 'Deaths':[0]*len(missing_days)}, 
                                index=missing_days)

df_covid = pd.concat([df_complementary, df_covid])
df_covid

,Country,Confirmed,Recovered,Deaths
2020-01-01,0,0,0,0
2020-01-02,0,0,0,0
2020-01-03,0,0,0,0
2020-01-04,0,0,0,0
2020-01-05,0,0,0,0
...,...,...,...,...
2020-04-23,US,869170,80203,49724
2020-04-24,US,905358,99079,51493
2020-04-25,US,938154,100372,53755
2020-04-26,US,965785,106988,54881


In [22]:
vix_url = 'https://datahub.io/core/finance-vix/r/vix-daily.csv'
df_vix = getDataset(vix_url)
df_vix = df_vix[df_vix['Date']>='2020-01-01'].copy()
df_vix.set_index('Date', inplace=True)
df_vix

,VIX Open,VIX High,VIX Low,VIX Close
Date,,,,
2020-01-02,13.46,13.72,12.42,12.47
2020-01-03,15.01,16.20,13.13,14.02
2020-01-06,15.45,16.39,13.54,13.85
2020-01-07,13.84,14.46,13.39,13.79
2020-01-08,15.16,15.24,12.83,13.45
...,...,...,...,...
2020-04-23,41.30,42.47,39.06,41.38
2020-04-24,41.91,42.08,35.60,35.93
2020-04-27,36.29,36.44,32.51,33.29


In [28]:
ngas_url = 'https://datahub.io/core/natural-gas/r/daily.csv'
df_ngas = getDataset(ngas_url)
df_ngas = df_ngas[df_ngas['Date']>='2020-01-01']
df_ngas.rename({'Price':'Natural Gas Price'}, axis=1, inplace=True)
df_ngas.set_index('Date', inplace=True)
df_ngas

,Natural Gas Price
Date,
2020-01-02,2.05
2020-01-03,2.06
2020-01-06,2.10
2020-01-07,2.17
2020-01-08,2.09
...,...
2020-04-22,1.90
2020-04-23,1.86
2020-04-24,1.81


In [36]:
import numpy as np

yields_url = 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=H15&series=fecc07f1fe124db4c69e29e9d70795fb&lastobs=&from=01/01/2020&to=04/30/2020&filetype=csv&label=include&layout=seriescolumn'
df_yields = getDataset(yields_url) #drop rows 2-6, rename columns
df_yields.drop(range(0,5), axis=0, inplace=True)
df_yields.replace('ND', np.NaN, inplace=True)
df_yields.columns = ['Date', 'Yield10y', 'Yield10y inflation indexed']
df_yields.set_index('Date', inplace=True)
df_yields

,Yield10y,Yield10y inflation indexed
Date,,
2020-01-01,NaN,NaN
2020-01-02,1.88,0.08
2020-01-03,1.80,0.03
2020-01-06,1.81,0.06
2020-01-07,1.83,0.09
...,...,...
2020-04-22,0.63,-0.43
2020-04-23,0.61,-0.44
2020-04-24,0.60,-0.51


In [72]:
# outer join all datasets on Date

merged = df_covid.join(df_vix, how='outer').join(df_ngas, how='outer').join(df_yields, how='outer')

merged['Country'] = 'US'
merged.fillna(method='ffill', inplace=True)
merged.fillna(method='bfill', inplace=True)
merged

,Country,Confirmed,Recovered,Deaths,VIX Open,VIX High,VIX Low,VIX Close,Natural Gas Price,Yield10y,Yield10y inflation indexed
2020-01-01,US,0.0,0.0,0.0,13.46,13.72,12.42,12.47,2.05,1.88,0.08
2020-01-02,US,0.0,0.0,0.0,13.46,13.72,12.42,12.47,2.05,1.88,0.08
2020-01-03,US,0.0,0.0,0.0,15.01,16.20,13.13,14.02,2.06,1.80,0.03
2020-01-04,US,0.0,0.0,0.0,15.01,16.20,13.13,14.02,2.06,1.80,0.03
2020-01-05,US,0.0,0.0,0.0,15.01,16.20,13.13,14.02,2.06,1.80,0.03
...,...,...,...,...,...,...,...,...,...,...,...
2020-04-25,US,938154.0,100372.0,53755.0,41.91,42.08,35.60,35.93,1.81,0.60,-0.51
2020-04-26,US,965785.0,106988.0,54881.0,41.91,42.08,35.60,35.93,1.81,0.60,-0.51
2020-04-27,US,988197.0,111424.0,56259.0,36.29,36.44,32.51,33.29,1.68,0.67,-0.50
2020-04-28,US,988197.0,111424.0,56259.0,33.21,35.39,30.54,33.57,1.80,0.62,-0.52


In [ ]:
# for plots

# https://stackoverflow.com/questions/18274137/how-to-animate-text-in-matplotlib
# https://stackoverflow.com/questions/7045729/automatically-position-text-box-in-matplotlib